In [44]:
include("StandardCode.jl")

function normalise_avgdot(df, genename, expn, typeprobe, ibidislide; root = pwd(), genefolder = "Gene")
    dot1_r1 = TSSs.int_brightest_pixel(TSSs.read_tiff_as_gray(root * typeprobe *"/_mRNA_AVG_ns.tif").*25.5; radious = 1)
    dot1_r2 = TSSs.int_brightest_pixel(TSSs.read_tiff_as_gray(root * typeprobe * "/_mRNA_AVG_ns.tif").*25.5; radious = 2)
    dot1_r3 = TSSs.int_brightest_pixel(TSSs.read_tiff_as_gray(root * typeprobe * "/_mRNA_AVG_ns.tif").*25.5; radious = 3)

    df[!,:TSS1_r1] = df[!,:locus1_int1_TSS2] ./ dot1_r1
    df[!,:TSS1_r2] = df[!,:locus1_int2_TSS2] ./ dot1_r2
    df[!,:TSS1_r3] = df[!,:locus1_int3_TSS2] ./ dot1_r3
    df[!,:TSS2_r1] = df[!,:locus2_int1_TSS2] ./ dot1_r1
    df[!,:TSS2_r2] = df[!,:locus2_int2_TSS2] ./ dot1_r2
    df[!,:TSS2_r3] = df[!,:locus2_int3_TSS2] ./ dot1_r3
    
    
    CSV.write("TSS_avgdot/"*genename*"_exp"*"$expn"*".csv", df)
    ibidislide[!,:Well] = [split(ii, " (")[1] for ii in ibidislide[!,:Well]]
    cp_dir = root1*"CP_results"
    cells1 = CellInfo(cp_dir, add_probetypes(ibidislide));
    exp1 = CSV.read("TSS_avgdot/"*gene*"_exp"*"$expn"*".csv", DataFrame)
    exp1[!,:Well] = [split(split(ii, "S 0_")[2], "_X")[1] for ii in exp1[!,:Image]]
    exp1 = innerjoin(exp1, ibidislide, on= "Well")
    CSV.write(genefolder*"/"*gene*"_exp"*"$expn"*"_TSS.csv", exp1)
    CSV.write(genefolder*"/"*gene*"_exp"*"$expn"*"_CP.csv", cells1)
    fq_dir1 = root1*typeprobe*"/"
    fq1 = FQ_summary_MATURE(fq_dir1)
    CSV.write(genefolder*"/"*gene*"_exp"*"$expn"*"_FQ.csv", fq1)
end

using ImageMagick
using ColorTypes
using FixedPointNumbers

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [70]:



function TSSs.TSS_raw_quant(t2, tss_folder, image_folder, n; xy = 0.189, zx = 0.5)
    
      images_pat = intersect(TSSs.get_image_patterns(t2), TSSs.get_image_patterns(tss_folder))
    
    p = Progress(length(images_pat), 1)

    dfs = []

    for a in 1:length(images_pat)
        next!(p)
        

        pat = images_pat[a]

        # Get all the TSS in the images
        a = TSSs.find_outline(tss_folder, pat)
          tss = TSSs.cell_tss_dict(a)
        
        tss = TSSs.meassure_tss(tss, imagesfolder, pat, n, xy = xy, zx = zx)
    
        df = TSSs.analysis_singlechannel(tss; image = pat,  tss_name = :TSS2)
        push!(dfs, df)

    end
    
    d = TSSs.join_in_all_common_columns(dfs...)
    return d
    
end

In [71]:
# Define where are the outlines

root3 = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp3/"

tss_c4 = root3 * "Segmentation_type1/_FQ_outline/_TS_detect"
imagesfolder = root3 * "/tiff3D"
images_pat = TSSs.get_image_patterns(tss_c4)
d= TSSs.TSS_raw_quant(tss_c4, tss_c4, imagesfolder, 4; xy = 0.189, zx = 0.5)

Progress: 100%|█████████████████████████████████████████| Time: 0:33:13


,Cell,Image
,String,SubStri…
1,Cell_CP_5,Exp3_CellMix_Ifit1_Mx1_15thNovember19_Sequence_A 0_S 0_U 1_V 0_X 6_Y14_Job(Seq.Scan) 1 01 01 02 01 04 02 01 01_074
2,Cell_CP_8,Exp3_CellMix_Ifit1_Mx1_15thNovember19_Sequence_A 0_S 0_U 1_V 0_X 6_Y14_Job(Seq.Scan) 1 01 01 02 01 04 02 01 01_074
3,Cell_CP_9,Exp3_CellMix_Ifit1_Mx1_15thNovember19_Sequence_A 0_S 0_U 1_V 0_X 6_Y14_Job(Seq.Scan) 1 01 01 02 01 04 02 01 01_074
4,Cell_CP_7,Exp3_CellMix_Ifit1_Mx1_15thNovember19_Sequence_A 0_S 0_U 1_V 0_X 6_Y14_Job(Seq.Scan) 1 01 01 02 01 04 02 01 01_074
5,Cell_CP_4,Exp3_CellMix_Ifit1_Mx1_15thNovember19_Sequence_A 0_S 0_U 1_V 0_X 6_Y14_Job(Seq.Scan) 1 01 01 02 01 04 02 01 01_074
6,Cell_CP_1,Exp3_CellMix_Ifit1_Mx1_15thNovember19_Sequence_A 0_S 0_U 1_V 0_X 6_Y14_Job(Seq.Scan) 1 01 01 02 01 04 02 01 01_074
7,Cell_CP_6,Exp3_CellMix_Ifit1_Mx1_15thNovember19_Sequence_A 0_S 0_U 1_V 0_X 6_Y14_Job(Seq.Scan) 1 01 01 02 01 04 02 01 01_074
8,Cell_CP_3,Exp3_CellMix_Ifit1_Mx1_15thNovember19_Sequence_A 0_S 0_U 1_V 0_X 6_Y14_Job(Seq.Scan) 1 01 01 02 01 04 02 01 01_074
9,Cell_CP_2,Exp3_CellMix_Ifit1_Mx1_15thNovember19_Sequence_A 0_S 0_U 1_V 0_X 6_Y14_Job(Seq.Scan) 1 01 01 02 01 04 02 01 01_074


In [7]:


count_tss_in_image(d)

26

In [ ]:

CSV.write("TSS_raw/Ifit1_exp3.csv", d)

In [ ]:
# Define where are the outlines

root2 = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp2/"

tss_c4 = root2 * "quick_analysis_qq/Segmentation_C4/_FQ_outline/_TS_detect"
imagesfolder = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp2/tiff3D"
images_pat = TSSs.get_image_patterns(tss_c4)


d= TSSs.TSS_raw_quant(tss_c4, tss_c4, imagesfolder, 4; xy = 0.189, zx = 0.5)

if !in("TSS_raw", readdir())
    mkdir("TSS_raw")
end

CSV.write("TSS_raw/Ifit1_exp2.csv", d)

In [ ]:
# Define where are the outlines

root1 = "/Volumes/lymphdev\$/SarahWells/Confocal/CellMix_Rad21KO_WT_Ifit1_Mx1/Exp1"

tss_c4 = root1 * "/Segmentation_type1/_FQ_outline/_TS_detect"
imagesfolder = root1 * "/tiff3D"
images_pat = TSSs.get_image_patterns(tss_c4)


d = TSSs.TSS_raw_quant(tss_c4, tss_c4, imagesfolder, 4; xy = 0.189, zx = 0.5)

if !in("TSS_raw", readdir())
    mkdir("TSS_raw")
end

CSV.write("TSS_raw/Ifit1_exp1.csv", d)